# As of 2024/08/30, this notebook has been superseded by the functions in the file gap_filling/ceds_derived_sectors.py

Created on: 2024/08/28

Created by: DPM


Summary: The purpose of this notebook is to use a combination of CEDS and EDGAR data to derive new sectors to partially replace the pre-2024 other-manufacturing CT sector. For a complete understanding of the methods and equations used to derive the new sectors see the following documents:

https://docs.google.com/spreadsheets/d/1Acei92ou6mZsOL2nXEOYNP7WPNH_A7Gds-AQ3GXdQDs/edit?gid=1870391791#gid=1870391791 ('2024 gap equations' Tab)

https://docs.google.com/presentation/d/1mHkGVHPjibjqEK2o6fG6Wlp7QnSpHzcmnTdHRliraZU/edit?usp=sharing

In [24]:
import argparse
import datetime

import pandas as pd
import numpy as np

from gap_filling.data_handler import DataHandler
from gap_filling.edgar_projection import ProjectData
from gap_filling.utils import (parse_and_format_data_to_insert, get_all_edgar_data, get_all_ceds_data, get_all_ceds_data_csv)
from gap_filling import annexI_food_bev


# def process_all(args, new_db):
############################
# Get the data
############################
# Init the Data Handler


# get connections
getedgar_conn = DataHandler()
getct_conn = DataHandler()
get_ceds_conn = DataHandler()
# write_conn = DataHandler()




In [25]:
edgar_data = get_all_edgar_data(getedgar_conn, get_projected=False)
#Convert column names to strings for processing
edgar_data.columns = edgar_data.columns.astype(str)


#Get CEDS data
# ceds_data = get_all_ceds_data(get_ceds_conn)
# #Convert column names to strings for processing
# ceds_data.columns = ceds_data.columns.astype(str)

ceds_data = get_all_ceds_data_csv()

#Now ensure all ceds and edgar values are numpy floats
for yr in range(2015,2023):
    edgar_data[str(yr)] = edgar_data[str(yr)].astype(float)
    ceds_data[str(yr)] = ceds_data[str(yr)].astype(float)
    

# Get the CT data from db just for reference
ct_data = getct_conn.load_data("climate-trace", years_to_columns=True)
ct_data.columns = ct_data.columns.astype(str)

## Below are individual code snippets to arrive at the ceds-derived data to be plugged into Christy's gap-filling equations

In [26]:
#Now create new CEDS sectors:
#First iron-and-steel (need to subtract CT steel)
add_secs = ['1A2a_Ind-Comb-Iron-steel', '2C1_Iron-steel-alloy-prod']
iron_steel_ceds_derived_data = ceds_data[ceds_data["Sector"].isin(add_secs)].groupby(["ID", "Gas"]).sum().reset_index()
iron_steel_ceds_derived_data["Sector"] = "iron-and-steel"



# Need to figure out why length of output dataframe in these steps does not match input dataframe

In [27]:
def sector_fractional_contribution(inventory_data, 
                            add_secs, 
                            inventory_data_contributing_sec, 
                            inventory_data_to_take_fraction_of, 
                            new_sector_name):
    """Calculates the fractional contribution of one 
    subsector to the sum of a list of sectors. Then
    applies that fractional contribution to a final 
    sector.

    Example: calculating the contribution of Glass to 
    to sum of Glass + Lime + Cement + other-minerals
    and applying that fraction to the combustion from
    those same sectors.

    Parameters
    ----------
    inventory_data: pandas df
    Country-level annual emissions by sector.
    Must include the sectors to be summed. Could be 
    edgar, ceds, faostat, climate-trace

    add_secs: list of strings
    list of sectors to be summed

    inventory_data_contributing_sec: pandas df
    Should be same inventory as inventory_data
    but is only for the sector(s) that you want
    to calculate the fractional contribution of.

    inventory_data_to_take_fraction_of: pandas df
    Can be from same or other inventory as inventory_data.
    Contains only one sector of which you would like to 
    attribute only a fraction to the new sector.

    new_sector_name: str
    name for the new sector which is the fraction of
    inventory_data_to_take_fraction_of contributed by
    inventory_data_contributing_sec sector.

    Returns
    -------
    frac_of_other_sector: pandas df
    inventory containing the new sector, no others.

    """
    
    inventory_data_sum_add_secs = inventory_data[inventory_data["Sector"].isin(add_secs)].groupby(["ID", "Gas"]).sum().reset_index()
    # t_edgar_total_sector_data["Sector"] = "cement-lime-glass-other"
    print('For testing... all add_secs sectors summed for the USA is:', \
          inventory_data_sum_add_secs.loc[inventory_data_sum_add_secs.ID == "USA",['Gas', '2020']])

    #Next add Glass-production so we can get its contribution to the total
    inventory_frac_subsector = pd.merge(inventory_data_sum_add_secs, inventory_data_contributing_sec, on=['ID', "Gas"], suffixes=('_total',''))
    for yr in range(2015,2023):
        inventory_frac_subsector[f'{yr}'] = inventory_frac_subsector[f'{yr}'] / inventory_frac_subsector[f'{yr}_total']
        inventory_frac_subsector.drop(columns=[f'{yr}_total'], inplace=True)
    inventory_frac_subsector.fillna(0, inplace=True)
    # edgar_frac_glass["Sector"] = "edgar-fraction-glass"
    inventory_frac_subsector.drop(columns=['Data source'], inplace=True)

    print('For testing... the fraction of the specified sector compared to the sum above for the USA is:', \
          inventory_frac_subsector.loc[inventory_frac_subsector.ID == "USA",['Gas', '2020']])


    #Now multiply edgar_frac_glass by 1.A.2.f to get the contribution of glass to 1.A.2.f
    frac_of_other_sector = pd.merge(inventory_frac_subsector, inventory_data_to_take_fraction_of, on=['ID', "Gas"], suffixes=('_frac','_total'))
    for yr in range(2015,2023):
        frac_of_other_sector[f'{yr}'] = frac_of_other_sector[f'{yr}_frac'] * frac_of_other_sector[f'{yr}_total']
        frac_of_other_sector.drop(columns=[f'{yr}_total', f'{yr}_frac'], inplace=True)
    frac_of_other_sector.fillna(0, inplace=True)
    frac_of_other_sector.drop(columns=['Sector_total', 'Sector_frac',
                                "Unit_frac", "Unit_total"], inplace=True)
    frac_of_other_sector["Sector"] = new_sector_name
    frac_of_other_sector["Unit"] = "tonnes"
    return frac_of_other_sector

In [28]:
#Now, need to handle proportions of sectors
#First get some of all sectors as we did above:
add_secs = ['2.A.3 Glass Production', '2.A.1 Cement production', '2.A.2 Lime production', \
            '2.A.4 Other Process Uses of Carbonates']


ceds_lime_comb = sector_fractional_contribution(edgar_data, add_secs, edgar_data[edgar_data['Sector'] =='2.A.2 Lime production'],
                        ceds_data[ceds_data['Sector'] == '1A2f_Ind-Comb-Non-metalic-minerals'],
                        'lime-combustion')


#Now check the work!
print("Total lime-production emissions: ", ceds_data.loc[(ceds_data['ID'] == "USA") & (ceds_data["Sector"] == '2A2_Lime-production'),['Gas', '2020']])
print("Total non-metalic-minerals combustion: ", ceds_data.loc[(ceds_data['ID'] == "USA") & (ceds_data["Sector"] == '1A2f_Ind-Comb-Non-metalic-minerals'),['Gas', '2020']])
print("Contribution of lime to non-metalic-minerals combustion: ", ceds_lime_comb.loc[ceds_lime_comb.ID=="USA",['Gas', '2020']])

print("\n If everything was successful, the following must be true:\n 1) the first number (for a given gas) times the second number (for a given gas) should equal the third number, \n 2) the final number should equal the product of the second and fourth numbers.")




For testing... all add_secs sectors summed for the USA is: year  Gas          2020
195   co2  6.648744e+07
For testing... the fraction of the specified sector compared to the sum above for the USA is: year  Gas      2020
108   co2  0.178229
Total lime-production emissions:         Gas          2020
11973  ch4  0.000000e+00
11973  co2  7.248908e+06
11973  n2o  0.000000e+00
Total non-metalic-minerals combustion:         Gas          2020
11950  ch4  7.960794e+02
11950  co2  3.477485e+07
11950  n2o  5.615269e+02
Contribution of lime to non-metalic-minerals combustion:       Gas          2020
107  co2  6.197892e+06

 If everything was successful, the following must be true:
 1) the first number (for a given gas) times the second number (for a given gas) should equal the third number, 
 2) the final number should equal the product of the second and fourth numbers.


In [29]:

#Now a slightly more complicated case where we have to employ EDGAR data to derive a fractional contribution of glass to CEDS
# 1A2f_Ind-Comb-Non-metalic-minerals
add_secs = ['2.A.3 Glass Production', '2.A.1 Cement production', '2.A.2 Lime production', \
            '2.A.4 Other Process Uses of Carbonates']

ceds_glass_comb = sector_fractional_contribution(edgar_data, add_secs, edgar_data[edgar_data['Sector'] =='2.A.3 Glass Production'],
                        ceds_data[ceds_data['Sector'] == '1A2f_Ind-Comb-Non-metalic-minerals'],
                        'glass-combustion')


'''I don't think this is needed since the gapfilling code fills in for missing gases

#Finally, we have to add 'n2o' and 'ch4' to this dataframe for each country because EDGAR assumes 0 emissions from these gases, while 
# CEDS does not.
def create_dummy_df(country_values, gas, sector, unit="tonnes"):
    dummy_df = pd.DataFrame({'ID': country_values})
    dummy_df['Gas'] = gas
    dummy_df['Unit'] = unit
    dummy_df['Sector'] = sector
    for c in np.arange(2015,2023).astype(str): dummy_df[c] = 0
    return dummy_df

n2o_dummy_df = create_dummy_df(np.unique(ceds_glass_comb.ID.values), 'n2o', 'glass-combustion')
ch4_dummy_df = create_dummy_df(np.unique(ceds_glass_comb.ID.values), 'ch4', 'glass-combustion')
ceds_glass_comb = pd.concat([ceds_glass_comb, n2o_dummy_df, ch4_dummy_df], axis=0).reset_index(drop=True)
'''

#Now check the work!
print("Total glass-production emissions: ", edgar_data.loc[(edgar_data['ID'] == "USA") & (edgar_data["Sector"] == '2.A.3 Glass Production'),['Gas', '2020']])
print("Total non-metalic-minerals combustion: ", ceds_data.loc[(ceds_data['ID'] == "USA") & (ceds_data["Sector"] == '1A2f_Ind-Comb-Non-metalic-minerals'),['Gas', '2020']])
print("Contribution of glass to non-metalic-minerals combustion: ", ceds_glass_comb.loc[ceds_glass_comb.ID=="USA",['Gas', '2020']])

print("\n If everything was successful, the following must be true:\n 1) the first number (for a given gas) times the second number (for a given gas) should equal the third number, \n 2) the final number should equal the product of the second and fourth numbers.")

#It seems like everything works!

For testing... all add_secs sectors summed for the USA is: year  Gas          2020
195   co2  6.648744e+07
For testing... the fraction of the specified sector compared to the sum above for the USA is: year  Gas      2020
195   co2  0.027083
Total glass-production emissions:  year  Gas         2020
6479  co2  1800675.072
Total non-metalic-minerals combustion:         Gas          2020
11950  ch4  7.960794e+02
11950  co2  3.477485e+07
11950  n2o  5.615269e+02
Contribution of glass to non-metalic-minerals combustion:       Gas           2020
192  co2  941804.979466

 If everything was successful, the following must be true:
 1) the first number (for a given gas) times the second number (for a given gas) should equal the third number, 
 2) the final number should equal the product of the second and fourth numbers.


In [30]:
## now need to make misc-mineral-industry-combustion ceds-derived sector to be included into misc CT sector
add_secs = ['2.A.3 Glass Production', '2.A.1 Cement production', '2.A.2 Lime production', \
            '2.A.4 Other Process Uses of Carbonates']

ceds_mineral_comb = sector_fractional_contribution(edgar_data, add_secs, edgar_data[edgar_data['Sector'] =='2.A.4 Other Process Uses of Carbonates'],
                        ceds_data[ceds_data['Sector'] == '1A2f_Ind-Comb-Non-metalic-minerals'],
                        'misc-mineral-industry-combustion')

#Now check the work!
print("Total other-mineral-production emissions: ", edgar_data.loc[(edgar_data['ID'] == "USA") & (edgar_data["Sector"] == '2.A.4 Other Process Uses of Carbonates'),['Gas', '2020']])
print("Total non-metalic-minerals combustion: ", ceds_data.loc[(ceds_data['ID'] == "USA") & (ceds_data["Sector"] == '1A2f_Ind-Comb-Non-metalic-minerals'),['Gas', '2020']])
print("Contribution of 'other' to non-metalic-minerals combustion: ", ceds_mineral_comb.loc[ceds_mineral_comb.ID=="USA",['Gas', '2020']])

print("\n If everything was successful, the following must be true:\n 1) the first number (for a given gas) times the second number (for a given gas) should equal the third number, \n 2) the final number should equal the product of the second and fourth numbers.")


For testing... all add_secs sectors summed for the USA is: year  Gas          2020
195   co2  6.648744e+07
For testing... the fraction of the specified sector compared to the sum above for the USA is: year  Gas      2020
173   co2  0.183084
Total other-mineral-production emissions:  year  Gas         2020
6664  co2  12172762.94
Total non-metalic-minerals combustion:         Gas          2020
11950  ch4  7.960794e+02
11950  co2  3.477485e+07
11950  n2o  5.615269e+02
Contribution of 'other' to non-metalic-minerals combustion:       Gas          2020
171  co2  6.366706e+06

 If everything was successful, the following must be true:
 1) the first number (for a given gas) times the second number (for a given gas) should equal the third number, 
 2) the final number should equal the product of the second and fourth numbers.


Test that misc_mineral_industry_combustion +cement_combustion + lime_combustion + glass-combustion equals non-metalic-minerals-combustion

In [31]:
#First get cement-combustion for completion:
add_secs = ['2.A.3 Glass Production', '2.A.1 Cement production', '2.A.2 Lime production', \
            '2.A.4 Other Process Uses of Carbonates']

ceds_cement_comb = sector_fractional_contribution(edgar_data, add_secs, edgar_data[edgar_data['Sector'] =='2.A.1 Cement production'],
                        ceds_data[ceds_data['Sector'] == '1A2f_Ind-Comb-Non-metalic-minerals'],
                        'cement-combustion')

#Combine all four dfs:
dfs_to_combine = [ceds_cement_comb, ceds_mineral_comb, ceds_glass_comb, ceds_lime_comb]
comb_df = dfs_to_combine[0]
suffixes = ["_cement", "_other", "_glass", "_lime"]
for i in range(1, len(dfs_to_combine)):
    comb_df = pd.merge(comb_df, dfs_to_combine[i], on=["ID", "Gas"], suffixes=('', suffixes[i]))
comb_df.columns = [col + suffixes[0] if col in np.arange(2015,2023).astype(str).tolist() else col\
                   for col in comb_df.columns]

#Now sum them
for yr in range(2015,2023):
    comb_df[str(yr)] = comb_df[f"{yr}_cement"] + comb_df[f"{yr}_glass"] +\
                    comb_df[f"{yr}_lime"] + comb_df[f"{yr}_other"]

comb_df = comb_df[np.arange(2015,2023).astype(str).tolist() + ["Gas"] + ["ID"]]

#Now merge with 1A2e CEDS data and take difference by year
ceds_comb_data = ceds_data[ceds_data["Sector"] == "1A2f_Ind-Comb-Non-metalic-minerals"]
merged_df = pd.merge(comb_df, ceds_comb_data, on=["ID", "Gas"], suffixes=('_summed', "_total"))

test_to_be_zero = []
for yr in range(2015,2023):
    test_to_be_zero.append(np.nansum(merged_df[f"{str(yr)}_summed"] - merged_df[f"{str(yr)}_total"]))

if all([x < 1e-5 for x in test_to_be_zero]):
    print("Testing complete... separate combustion emissions sum to total CEDS 1A2f combustion emissions.")

For testing... all add_secs sectors summed for the USA is: year  Gas          2020
195   co2  6.648744e+07
For testing... the fraction of the specified sector compared to the sum above for the USA is: year  Gas      2020
156   co2  0.611604
Testing complete... separate combustion emissions sum to total CEDS 1A2f combustion emissions.


### We will be doing something slightly different for food-beverage-tobacco. We have decided to take the Annex I countries that have reported both 2.H.2 and 1.A.2.e emissions (n=6 countries), quantified the country-specific relationship between these two (either mean of the annual ratios, or the slope of annual values) and applied this relationship to CEDS 1.A.2.e emissions for these 6 countries to estimate their 2.H.2 emissions. We will assume 0 emissions for all other countries.

For the calculation of the country-specific EFs, see the food_bev_pulp_paper_estimates.ipynb notebook. We will read these as a csv output by functions developed in that notebook.

In [32]:
food_bev_direct = annexI_food_bev.main()

/Users/dpmoore/Code/climate-trace-gapfilling/gap_filling/annexI_food_bev.py:113: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sector_ceds_df.loc[:, "Sector"] = "2.H.2-food-beverage-and-tobacco-direct"


## Now we can combine everything into one dataframe for inserting into the db

In [35]:
ceds_derived_df = pd.concat([ceds_mineral_comb,
                             ceds_lime_comb, ceds_glass_comb,
                             food_bev_direct])
ceds_derived_df['Unit'] = 'tonnes'
ceds_derived_df['Data source'] = 'ceds-derived'
ceds_derived_df = ceds_derived_df.sort_values(by=['ID', 'Sector','Gas']).reset_index(drop=True)

In [36]:
#Need to convert columns to integers
def convert_column_to_int(col):
    try:
        return(int(col))
    except ValueError:
        return col
ceds_derived_df.rename(columns=convert_column_to_int, inplace=True)

data_to_insert = parse_and_format_data_to_insert(ceds_derived_df)
data_to_insert['created_date'] = datetime.datetime.now().isoformat()